# Deep Water Bathymetric Particle Filter Experiment

This notebook runs the experiment testing the capabilities of the particle filter to conduct navigation using deep water bathymetry and for long duration.

## Data set preparation

First need to process the .m77t files in `source_data` into our database format.

In [ ]:
from src import process_dataset as pdset
import os
import pandas as pd

### Preprocess tracklines into sql/df format

In [ ]:
# Check and see if the .db directory exists
if not os.path.exists(".db"):
    os.mkdir(".db")
    
# Check to see if the database exists
if not os.path.exists('.db/tracklines.db'):
    tables = []
else:
    tables = pdset.get_tables('.db/tracklines.db')

source_data_location = "./source_data/"
# walk through the .m77t files in the source_data directory
for root, dirs, files in os.walk(source_data_location):
    for file in files:
        if file.endswith(".m77t"):
            # check to see if the file has already been processed
            filename = os.path.splitext(file)[0]
            if filename not in tables:
                print("Processing file: " + file)
                data = pd.read_csv(os.path.join(root, file), delimiter="\t", header=0)
                data = pdset.m77t_to_df(data)
                # data.to_sql(
                #    filename, sqlite3.connect(".db/tracklines.db"), if_exists="replace"
                # )
                pdset.save_dataset(
                    [data],
                    [filename],
                    output_location=".db",
                    output_format="db",
                    dataset_name="tracklines",
                )
            else:
                print("Skipping file: " + file + " (already processed)")

### Parse the raw data into tracklines of continuous data collections

In [ ]:
max_time = 10 # minutes
max_delta_t = 2 # minutes between points
min_duration = 60 # minutes, minimum duration for a useful trackline

data, names = pdset.parse_tracklines_from_db(
    ".db/tracklines.db",
    #max_time,
    #max_delta_t,
    #min_duration,
    data_types=[
        "bathy",
        "mag",
        "grav",
        ["bathy", "mag"],
        ["bathy", "grav"],
        ["grav", "mag"],
        ["bathy", "grav", "mag"],
    ],
)
# Save the parsed data to the database
pdset.save_dataset(
    data, names, output_location=".db", output_format="db", dataset_name="parsed"
)
#summary = pdset.get_parsed_data_summary(data, names)

## Simulation parameters verification

First we need to tune the particle filter propagation noise to be similar to that of a marine-grade inertial navigation system. A low-end marine-grade INS should have a drift of 1 nm per 24 hours.

In [ ]:
from src.particle_filter import rmse, propagate
import numpy as np

time = 24*60 # minutes
noise = np.array([0, 2.6, 0])
bound = 1852 # meters

errors = []
for v in range(1, 26):
    P = np.asarray([[0,0,0,0,v,0]])
    T = P.copy()
    t = 0
    for i in range(50000):
        # Eastward
        u = [0,v,0]
        while t < time:    
            P = propagate(P, u, noise=np.diag(noise), noise_calibration_mode=True)
            T = propagate(T, u, noise=np.diag([0,0,0]), noise_calibration_mode=False)
            t+=1
        errors.append(rmse(P, T[0, :2]))
        # Northward
        u = [v,0,0]
        while t < time:    
            P = propagate(P, u, noise=np.diag(noise), noise_calibration_mode=True)
            T = propagate(T, u, noise=np.diag([0,0,0]), noise_calibration_mode=False)
            t+=1
        errors.append(rmse(P, T[0, :2]))
        # Northeastward 
        u = np.array([1,1,0]) / np.linalg.norm([1,1,0])
        u *= v
        while t < time:    
            P = propagate(P, u, noise=np.diag(noise), noise_calibration_mode=True)
            T = propagate(T, u, noise=np.diag([0,0,0]), noise_calibration_mode=False)
            t+=1
        errors.append(rmse(P, T[0, :2]))

print(f"RMSE: {np.mean(errors)}")

In [ ]:
from matplotlib import pyplot as plt

if not os.path.exists(".db/plots"):
    os.makedirs(".db/plots")

plt.hist(errors, bins=25, density=True)
plt.xlabel("RMSE (m)")
plt.ylabel("Frequency")
plt.title("RMSE of Particle Filter")
plt.savefig(".db/plots/propagation_tuning.png")
plt.show()

In [ ]:
config = {"velocity_noise": [noise[1], noise[1], 0]}

Next we need to develop the measurement value standard deviation. We'll first do some general examination of the data. Namely, investigating the sensor measurements to see if we can build a reasonable sensor model.

In [ ]:
from src.gmt_tool import inflate_bounds, get_map_section, get_map_point
import numpy as np

tables = pdset.get_tables(".db/parsed.db")
bathy_tables = [table for table in tables if "_D_" in table]

d_bathy = np.array([])

for table in bathy_tables:
    data = pdset.table_to_df(".db/parsed.db", table)
    min_lon = data.LON.min()
    max_lon = data.LON.max()
    min_lat = data.LAT.min()
    max_lat = data.LAT.max()
    min_lon, min_lat, max_lon, max_lat = inflate_bounds(min_lon, min_lat, max_lon, max_lat, 0.25)
    bathy_map = get_map_section(min_lon, max_lon, min_lat, max_lat, 'relief', '15s', "temp")
    d_bathy = np.hstack([d_bathy, data['DEPTH'] - (-get_map_point(bathy_map, data.LON, data.LAT))])

config['bathy_mean_d'] = np.mean(d_bathy, where=~np.isnan(d_bathy))
config['bathy_std'] = np.std(d_bathy, where=~np.isnan(d_bathy))

plt.hist(d_bathy, bins=100, density=True)
plt.xlim([-250,250])
plt.xlabel("Depth Difference (m)")
plt.ylabel("Frequency")
plt.title("Bathymetry Difference")
plt.savefig(".db/plots/bathy_diff.png")
plt.show()

In [ ]:
import json

with open("config.json", "w") as f:
    json.dump(config, f)

### Populate with velocities

In [1]:
from src.particle_filter import process_particle_filter, populate_velocities, plot_error, plot_estimate, summarize_results
import json
import src.process_dataset as pdset
from tqdm import tqdm
import pandas as pd
import os
import matplotlib.pyplot as plt

In [2]:
tables = pdset.get_tables(".db/parsed.db")
bathy_tables = [table for table in tables if "_D_" in table]
config = json.load(open("config.json", "r"))
config["n"] = 1000
config["cov"] = [1/60, 1/60, 0, config["velocity_noise"][0], config["velocity_noise"][1], 0]

with open("config.json", "w") as f:
    json.dump(config, f)

In [3]:
config

{'velocity_noise': [2.6, 2.6, 0],
 'bathy_mean_d': 21.285045267468078,
 'bathy_std': 215.01077619593897,
 'n': 1000,
 'cov': [0.016666666666666666, 0.016666666666666666, 0, 2.6, 2.6, 0]}

In [4]:
df = pdset.table_to_df(".db/parsed.db", bathy_tables[3])

In [5]:
len(df)

47

In [6]:
if not os.path.exists('.db/plots2/estimate/'):
    os.makedirs('.db/plots2/estimate/')

if not os.path.exists('.db/plots2/errors/'):
    os.makedirs('.db/plots2/errors/')

summary = None
annotations = {"recovery": 1852, "res":1852/4}
# for table in tqdm(bathy_tables):
#     print(f"Running {table}")
#     df = pdset.table_to_df(".db/parsed.db", table)
    
df = populate_velocities(df)
results, geo_map = process_particle_filter(df, config)
print("Run complete! Saving results...")
pdset.save_dataset(
    [results], ["test"], output_location=".db", output_format="db", dataset_name="results"
)


Run complete! Saving results...


In [7]:
print("Results saved! Plotting...")
fig, ax = plot_estimate(geo_map, results)
fig.savefig(f".db/plots2/estimate/test_estimate.png")
plt.close(fig)
fig, ax = plot_error(results, annotations=annotations)
fig.savefig(f".db/plots2/errors/test_error.png")
plt.close(fig)

Results saved! Plotting...


In [ ]:
summary = None
results_tables = pdset.get_tables(".db/results.db")
for table in results_tables:
    df = pdset.table_to_df(".db/results.db", table)
    run = summarize_results(df, 1852)
    run["Name"] = table
    if summary is None:
        summary = run#.copy()
    else:
        summary = pd.concat([summary, run], ignore_index=True)
    summary.to_csv(".db/summary_recovery.csv")

In [ ]:
summary = None
results_tables = pdset.get_tables(".db/results.db")
for table in results_tables:
    df = pdset.table_to_df(".db/results.db", table)
    run = summarize_results(df, 452)
    run["Name"] = table
    if summary is None:
        summary = run#.copy()
    else:
        summary = pd.concat([summary, run], ignore_index=True)
    summary.to_csv(".db/summary_resolution.csv")